In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [2]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# check null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# Preprocessing

# Handling null values in train data

In [4]:
# Drop not required columns and missing values
df = df.drop(['Alley', 'MiscFeature', 'Fence', 'PoolQC', 'MasVnrType'], axis='columns')

In [5]:
df = df.fillna({
    'LotFrontage': df['LotFrontage'].mean(),
    'MasVnrArea': df['MasVnrArea'].mean(),
    'BsmtFinSF1': df['BsmtFinSF1'].mean()
})


In [6]:
df = df.fillna({
    'BsmtQual': df['BsmtQual'].mode()[0],
    'BsmtCond': df['BsmtCond'].mode()[0],
    'BsmtExposure': df['BsmtExposure'].mode()[0],
    'BsmtFinType1': df['BsmtFinType1'].mode()[0],
    'BsmtFinType2': df['BsmtFinType2'].mode()[0],
    'Electrical': df['Electrical'].mode()[0],
    'FireplaceQu': df['FireplaceQu'].mode()[0]
})

In [7]:
# Handle null values for Garage properties
df = df.fillna({
    'GarageYrBlt': df['YearBuilt'],
    'GarageFinish': df['GarageFinish'].mode()[0],
    'GarageQual': df['GarageQual'].mode()[0],
    'GarageCond': df['GarageCond'].mode()[0],
    'GarageType': df['GarageType'].mode()[0]
})

In [8]:
df.shape

(1460, 76)

In [9]:
# check null values again
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [10]:
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [11]:
# Drop not required columns and missing values
df_test = df_test.drop(['Alley', 'MiscFeature', 'Fence', 'PoolQC', 'MasVnrType'], axis='columns')


In [12]:
df_test.shape

(1459, 75)

# Handling null values in test data

In [13]:
df_test = df_test.fillna({
    'LotFrontage': df_test['LotFrontage'].mean(),
    'MasVnrArea': df_test['MasVnrArea'].mean(),
    'BsmtFinSF1': df_test['BsmtFinSF1'].mean(),
    'BsmtFullBath': df_test['BsmtFullBath'].mean(),
    'BsmtHalfBath': df_test['BsmtHalfBath'].mean(),   
    'BsmtFinSF2': df_test['BsmtFinSF2'].mean(),     
    'BsmtUnfSF': df_test['BsmtUnfSF'].mean(),      
    'TotalBsmtSF': df_test['TotalBsmtSF'].mean()
})

In [14]:
df_test = df_test.fillna({
    'BsmtQual': df_test['BsmtQual'].mode()[0],
    'BsmtCond': df_test['BsmtCond'].mode()[0],
    'BsmtExposure': df_test['BsmtExposure'].mode()[0],
    'BsmtFinType1': df_test['BsmtFinType1'].mode()[0],
    'BsmtFinType2': df_test['BsmtFinType2'].mode()[0],
    'Electrical': df_test['Electrical'].mode()[0],
    'FireplaceQu': df_test['FireplaceQu'].mode()[0],
    'Utilities': df_test['Utilities'].mode()[0],
    'SaleType': df_test['SaleType'].mode()[0],
    'KitchenQual': df_test['KitchenQual'].mode()[0],
    'Exterior1st': df_test['Exterior1st'].mode()[0],
    'Exterior2nd': df_test['Exterior2nd'].mode()[0],
    'Functional': df_test['Functional'].mode()[0],
    'MSZoning': df_test['MSZoning'].mode()[0]
})

In [15]:
# Handle null values for Garage properties
df_test = df_test.fillna({
    'GarageYrBlt': df_test['YearBuilt'],
    'GarageFinish': df_test['GarageFinish'].mode()[0],
    'GarageQual': df_test['GarageQual'].mode()[0],
    'GarageCond': df_test['GarageCond'].mode()[0],
    'GarageType': df_test['GarageType'].mode()[0],
    'GarageCars': df_test['GarageCars'].mean(),
    'GarageArea': df_test['GarageArea'].mean()
})

In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1459 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [17]:
df_combined = pd.concat([df, df_test], axis='rows')

In [18]:
df_combined.shape

(2919, 76)

In [19]:
df_combined.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenP

# Handling categorical columns

In [20]:
def category_onehot_multcols(multcolumns):
    df_final=df_combined
    i=0
    for fields in multcolumns: 
        print(fields)
        df1=pd.get_dummies(df_combined[fields],drop_first=True, dtype=int)
        
        df_combined.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1  
        
    df_final=pd.concat([df_combined,df_final],axis=1)
        
    return df_final

In [21]:
df_combined['SalePrice']

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1454         NaN
1455         NaN
1456         NaN
1457         NaN
1458         NaN
Name: SalePrice, Length: 2919, dtype: float64

In [22]:
categorical_columns=['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood',
         'Condition2','BldgType','Condition1','HouseStyle','SaleType',
        'SaleCondition','ExterCond',
         'ExterQual','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
        'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','Heating','HeatingQC',
         'CentralAir',
         'Electrical','KitchenQual','Functional',
         'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive']

In [23]:
final_df=category_onehot_multcols(categorical_columns)

MSZoning
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition2
BldgType
Condition1
HouseStyle
SaleType
SaleCondition
ExterCond
ExterQual
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive


In [24]:
final_df.shape

(2919, 235)

# Remove duplicates

In [25]:
final_df =final_df.loc[:,~final_df.columns.duplicated()]

In [26]:
final_df.shape

(2919, 177)

In [27]:
df_train= final_df.iloc[:1460, :]
df_test = final_df.iloc[1460:, :]

In [28]:
df_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,Min1,Min2,Typ,Attchd,Basment,BuiltIn,CarPort,Detchd,RFn,P
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,1,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,1,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,1,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,1,0,0,0,0,1,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,1,1,0,0,0,0,1,0


In [29]:
df_test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,Min1,Min2,Typ,Attchd,Basment,BuiltIn,CarPort,Detchd,RFn,P
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,1,1,0,0,0,0,0,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,1,1,0,0,0,0,0,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,1,1,0,0,0,0,0,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,1,1,0,0,0,0,0,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,1,1,0,0,0,0,1,0


In [30]:
df_test.drop(['SalePrice'], axis='columns',inplace=True)

/tmp/ipykernel_18/1171851223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(['SalePrice'], axis='columns',inplace=True)


# Modeling

In [31]:
X_train = df_train.drop(['SalePrice'], axis='columns')
y_train = df_train['SalePrice']

In [32]:
import xgboost
classifier=xgboost.XGBRegressor()
classifier.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [33]:
y_pred = classifier.predict(df_test)

In [34]:
classifier.score(df_test, y_pred)

1.0

# Hyperparameter tuning

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
clf = GridSearchCV(
    estimator = xgboost.XGBRegressor(), 
    param_grid= {
            'max_depth': [3, 6, 9],
            'min_child_weight': [1, 10, 100],
            'learning_rate': [0.05, 0.1, 0.20]
    },
    cv=5, 
    return_train_score=True
)

In [37]:
scores = []
clf.fit(X_train, y_train)
scores.append(
   { 
    'best_score': clf.best_score_,
    'best_params': clf.best_params_
   }
)
df_score = pd.DataFrame(scores, columns=['best_score', 'best_params'])
df_score

,best_score,best_params
0,0.884859,"{'learning_rate': 0.2, 'max_depth': 3, 'min_ch..."


In [38]:
tuned_classifier=xgboost.XGBRegressor(max_depth=3, min_child_weight=1)
tuned_classifier.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [39]:
y_pred = tuned_classifier.predict(df_test)

In [40]:
prediction= pd.DataFrame(y_pred)
submission_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
datasets = pd.concat([submission_df['Id'], prediction], axis='columns')
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('sample_submission.csv', index=False)